## Importing Data and Libraries

In [11]:
import pandas as pd
import numpy as np

df_participant = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/dqthon-participants.csv')
df_participant.head()

,participant_id,first_name,last_name,birth_date,address,phone_number,country,institute,occupation,register_time
0,bd9b6f88-b84f-4c4d-90f8-b67fe2f1a29a,Citra,Nurdiyanti,05 Feb 1991,"Gg. Monginsidi No. 08\nMedan, Aceh 80734",(0151) 081 2706,Georgia,UD Prakasa Mandasari,Business Intelligence Engineer,1.617634e+09
1,7dfe3391-6f40-47b6-b4db-0c76ebaf5fc3,Aris,Setiawan,11 Jan 1993,"Gg. Rajawali Timur No. 7\nPrabumulih, MA 09434",+62 (036) 461 7027,Korea Utara,Universitas Diponegoro,Frontend Engineer,1.617634e+09
2,19582d7f-b824-4fe5-a517-d5bf573fc768,Cornelia,Handayani,31 Jul 1993,"Jalan Kebonjati No. 0\nAmbon, SS 57739",089 833 6695,Komoro,UD Hardiansyah Puspasari,Business Analyst,1.617634e+09
3,aeb6d817-25f3-4867-8a74-8d92e0a0f633,Soleh,Rajasa,04 Nov 1991,"Jl. Yos Sudarso No. 109\nLubuklinggau, SR 76156",+62 (418) 329-4756,Eritrea,Perum Tampubolon Yuliarti,DevOps Engineer,1.617634e+09
4,1fdabdd9-5444-4c97-87b2-fe8833ad0d27,Vivi,Astuti,22 Jan 2003,"Jalan Gardujati No. 53\nKediri, Sulawesi Tenga...",0812511835,Aljazair,PT Hardiansyah Rahimah,Data Analyst,1.617634e+09


## Transforming and Cleaning Data

Logistic team need postal code, city, github, email data etc for delivery needs. So we will transform the Postal Code first. What we know is, theres postal code data in address column.

In [12]:
df_participant['postal_code'] = df_participant['address'].str.extract(r'(\d+)$') 
df_participant['postal_code'].head()

0    80734
1    09434
2    57739
3    76156
4    70074
Name: postal_code, dtype: object

In [13]:
df_participant['city'] = df_participant['address'].str.extract(r'(?<=\n)(\w.+)(?=,)') 
df_participant['city'].head()

0           Medan
1      Prabumulih
2           Ambon
3    Lubuklinggau
4          Kediri
Name: city, dtype: object

In [14]:
df_participant['github_profile'] = 'https://github.com/' + df_participant['first_name'].str.lower() + df_participant['last_name'].str.lower()
df_participant['github_profile'].head()

0      https://github.com/citranurdiyanti
1         https://github.com/arissetiawan
2    https://github.com/corneliahandayani
3          https://github.com/solehrajasa
4           https://github.com/viviastuti
Name: github_profile, dtype: object

In [16]:
#Change +62 or 62 in the beginning to 0 because those number and symbol are Indonesia's phone number code
df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0')
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '')
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'\s+', '')
df_participant['cleaned_phone_number'].head()

C:\Users\Rakhmad Fajar\AppData\Local\Temp\ipykernel_19672\175390682.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0')
C:\Users\Rakhmad Fajar\AppData\Local\Temp\ipykernel_19672\175390682.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '')
C:\Users\Rakhmad Fajar\AppData\Local\Temp\ipykernel_19672\175390682.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'\s+', '')


0    01510812706
1    00364617027
2     0898336695
3    04183294756
4     0812511835
Name: cleaned_phone_number, dtype: object

In [20]:
#Make new column called Team Name that consist of firstname, lastname and institution.
def func(col):
    abbrev_name = "%s%s"%(col['first_name'][0],col['last_name'][0]) #Singkatan dari Nama Depan dan Nama Belakang dengan mengambil huruf pertama
    country = col['country']
    abbrev_institute = '%s'%(''.join(list(map(lambda word: word[0], col['institute'].split())))) #Singkatan dari value di kolom institute
    return "%s-%s-%s"%(abbrev_name,country,abbrev_institute)

df_participant['team_name'] = df_participant.apply(func, axis=1)
df_participant['team_name'].head()

0       CN-Georgia-UPM
1    AS-Korea Utara-UD
2        CH-Komoro-UHP
3       SR-Eritrea-PTY
4      VA-Aljazair-PHR
Name: team_name, dtype: object

In [21]:
#Making email column
def func(col):
    first_name_lower = col['first_name'].lower()
    last_name_lower = col['last_name'].lower()
    institute = ''.join(list(map(lambda word: word[0], col['institute'].lower().split()))) #Singkatan dari nama perusahaan dalam lowercase

    if 'Universitas' in col['institute']:
        if len(col['country'].split()) > 1: #Kondisi untuk mengecek apakah jumlah kata dari country lebih dari 1
            country = ''.join(list(map(lambda word: word[0], col['country'].lower().split())))
        else:
            country = col['country'][:3].lower()
        return "%s%s@%s.ac.%s"%(first_name_lower,last_name_lower,institute,country)

    return "%s%s@%s.com"%(first_name_lower,last_name_lower,institute)

df_participant['email'] = df_participant.apply(func, axis=1)

df_participant['email'].head()

0      citranurdiyanti@upm.com
1        arissetiawan@ud.ac.ku
2    corneliahandayani@uhp.com
3          solehrajasa@pty.com
4           viviastuti@phr.com
Name: email, dtype: object

In [24]:
#Making birth date column
#04 Nov 1991 to SQL Format (YY/MM/DD)
df_participant['birth_date'] = pd.to_datetime(df_participant['birth_date'], format='%d %b %Y')
df_participant['birth_date'].head()

0   1991-02-05
1   1993-01-11
2   1993-07-31
3   1991-11-04
4   2003-01-22
Name: birth_date, dtype: datetime64[ns]

In [25]:
#changing register date to SQL Format
df_participant['register_at'] = pd.to_datetime(df_participant['register_time'], unit='s')
df_participant['register_at'].head()

0   2021-04-05 14:47:26
1   2021-04-05 14:46:58
2   2021-04-05 14:47:15
3   2021-04-05 14:47:14
4   2021-04-05 14:46:50
Name: register_at, dtype: datetime64[ns]
